# Edge Case Testing
This notebook tests edge cases and error handling for `qutePandas` against `pandas` baselines.

In [1]:
import os
import sys
import importlib
import pandas as pd
import numpy as np
import pykx as kx
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('.'))
import qutePandas as qpd
importlib.reload(qpd)
from test_utils import verify_correctness
local_lic = os.path.abspath('../kdb_lic')
if os.path.exists(local_lic): os.environ['QLIC'] = local_lic
qpd.connect()
print('Setup Complete')


/Users/isha/Desktop/Projects/qutePandas/venvQutePandas/lib/python3.10/site-packages/pykx/config.py:112: UserWarning: Configuration value QLIC set to non directory value: /Users/isha/Desktop/Projects/qutePandas/kdb_lic
  warn(f'Configuration value QLIC set to non directory value: {_qlic}')


Setup Complete


## Create Test DataFrame
Define a small DataFrame with realistic mixed-type data including null values.

In [2]:
# Pandas DataFrame with mixed dtypes
# Expected: Table is created with correct row count
df = pd.DataFrame({
    "a": [1, 2, 3],
    "b": ["x", "y", "z"]
})

q_df = qpd.DataFrame(df)
qpd.print(q_df)

assert isinstance(q_df, kx.Table)
assert q_df.shape[0] == 3


┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ x │
│ 2 │ y │
│ 3 │ z │
└───┴───┘


In [3]:
# Pandas DataFrame with nulls and NaT
# Expected: Table is created and nulls are preserved
df = pd.DataFrame({
    "a": [1, None, 3],
    "b": [pd.Timestamp("2020-01-01"), pd.NaT, pd.Timestamp("2022-01-01")]
})

q_df = qpd.DataFrame(df)
qpd.print(q_df)

assert isinstance(q_df, kx.Table)
assert q_df.shape[0] == 3


┌─────┬─────────────────────┐
│ a   │ b                   │
├─────┼─────────────────────┤
│ 1.0 │ 2020-01-01 00:00:00 │
│ nan │ NaT                 │
│ 3.0 │ 2022-01-01 00:00:00 │
└─────┴─────────────────────┘


In [4]:
# Empty Pandas DataFrame
# Expected: Empty table with zero rows
df_empty = pd.DataFrame(columns=["a", "b"])

q_empty = qpd.DataFrame(df_empty)
qpd.print(q_empty)

assert isinstance(q_empty, kx.Table)
assert q_empty.shape[0] == 0


┌───┬───┐
│ a │ b │
├───┼───┤
└───┴───┘


In [5]:
# Dictionary input
# Expected: Table is created with matching rows
data = {"a": [1, 2], "b": [3, 4]}

q_df = qpd.DataFrame(data)
qpd.print(q_df)

assert isinstance(q_df, kx.Table)
assert q_df.shape[0] == 2


┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ 3 │
│ 2 │ 4 │
└───┴───┘


In [6]:
# Dictionary with unequal column lengths
# Expected: RuntimeError is raised
data = {"a": [1, 2], "b": [3]}

try:
    qpd.DataFrame(data)
    raise AssertionError("Expected RuntimeError")
except RuntimeError:
    pass


In [7]:
# Dictionary with unequal column lengths
# Expected: RuntimeError is raised
data = {"a": [1, 2], "b": [3]}

try:
    qpd.DataFrame(data)
    raise AssertionError("Expected RuntimeError")
except RuntimeError:
    pass


In [8]:
# List of lists with explicit columns
# Expected: Table is created with correct column count
data = [[1, "x"], [2, "y"], [3, "z"]]

q_df = qpd.DataFrame(data, columns=["a", "b"])
qpd.print(q_df)

assert isinstance(q_df, kx.Table)
assert q_df.shape[1] == 2


┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ x │
│ 2 │ y │
│ 3 │ z │
└───┴───┘


In [9]:
# List of lists without columns
# Expected: Columns are auto-generated
data = [[1, 2], [3, 4]]

q_df = qpd.DataFrame(data)
qpd.print(q_df)

assert isinstance(q_df, kx.Table)
assert q_df.shape[1] == 2


┌───────┬───────┐
│ col_0 │ col_1 │
├───────┼───────┤
│ 1     │ 2     │
│ 3     │ 4     │
└───────┴───────┘


In [10]:
# Empty list input
# Expected: Empty table is created
q_df = qpd.DataFrame([])
qpd.print(q_df)

assert isinstance(q_df, kx.Table)
assert q_df.shape[0] == 0


┌──────┐
│ data │
├──────┤
└──────┘


In [11]:
# Generator input
# Expected: Generator is materialized into table
gen = (i for i in range(5))

q_df = qpd.DataFrame(gen)
qpd.print(q_df)

assert isinstance(q_df, kx.Table)
assert q_df.shape[0] == 5


┌──────┐
│ data │
├──────┤
│ 0    │
│ 1    │
│ 2    │
│ 3    │
│ 4    │
└──────┘


In [12]:
# Scalar input
# Expected: Scalar is wrapped into single-row table
q_df = qpd.DataFrame(42)
qpd.print(q_df)

assert isinstance(q_df, kx.Table)
assert q_df.shape[0] == 1


┌──────┐
│ data │
├──────┤
│ 42   │
└──────┘


In [13]:
# Existing kx.Table input
# Expected: Table is returned unchanged
t = kx.q("([] a:1 2 3; b:4 5 6)")

q_df = qpd.DataFrame(t)
qpd.print(q_df)

assert q_df is t


┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ 4 │
│ 2 │ 5 │
│ 3 │ 6 │
└───┴───┘


In [14]:
# Keyed table input
# Expected: Keyed table is accepted
kt = kx.q("([k:1 2 3] v:10 20 30)")

q_df = qpd.DataFrame(kt)
assert isinstance(q_df, kx.KeyedTable)


In [15]:
# Invalid input type
# Expected: RuntimeError is raised
try:
    qpd.DataFrame(object())
    raise AssertionError("Expected RuntimeError")
except RuntimeError:
    pass


In [16]:
# Column length mismatch
# Expected: RuntimeError is raised
data = [[1, 2, 3]]

try:
    qpd.DataFrame(data, columns=["a", "b"])
    raise AssertionError("Expected RuntimeError")
except RuntimeError:
    pass


## Cleaning Functions
Test data cleaning operations on the mixed-type DataFrame.

In [17]:
# dropna_col on Pandas DataFrame
# Expected: Rows with nulls in specified column are removed
df = pd.DataFrame({
    "a": [1, None, 3],
    "b": [10, 20, 30]
})

q_df = qpd.DataFrame(df)
q_res = qpd.dropna_col(q_df, "a", return_type="q")

qpd.print(q_res)

pd_res = df.dropna(subset=["a"])
assert verify_correctness(pd_res, qpd.dropna_col(q_df, "a", return_type="p"))


┌─────┬────┐
│ a   │ b  │
├─────┼────┤
│ 1.0 │ 10 │
│ 3.0 │ 30 │
└─────┴────┘


In [18]:
# dropna_col where no nulls exist
# Expected: Table remains unchanged
df = pd.DataFrame({
    "a": [1, 2, 3],
    "b": [4, 5, 6]
})

q_df = qpd.DataFrame(df)
q_res = qpd.dropna_col(q_df, "a", return_type="q")

qpd.print(q_res)

pd_res = df.dropna(subset=["a"])
assert verify_correctness(pd_res, qpd.dropna_col(q_df, "a", return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ 4 │
│ 2 │ 5 │
│ 3 │ 6 │
└───┴───┘


In [19]:
# dropna_col on empty table
# Expected: Empty table is returned
df = pd.DataFrame(columns=["a", "b"])

q_df = qpd.DataFrame(df)
q_res = qpd.dropna_col(q_df, "a", return_type="q")

qpd.print(q_res)

pd_res = df.dropna(subset=["a"])
assert verify_correctness(pd_res, qpd.dropna_col(q_df, "a", return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
└───┴───┘


In [20]:
# dropna_col on column with all nulls
# Expected: Empty table is returned
df = pd.DataFrame({
    "a": [None, None],
    "b": [1, 2]
})

q_df = qpd.DataFrame(df)
q_res = qpd.dropna_col(q_df, "a", return_type="q")

qpd.print(q_res)

pd_res = df.dropna(subset=["a"])
assert verify_correctness(pd_res, qpd.dropna_col(q_df, "a", return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
└───┴───┘


In [21]:
# dropna removing any row with at least one null
# Expected: Rows containing any null are removed
df = pd.DataFrame({
    "a": [1, None, 3],
    "b": [10, 20, None]
})

q_df = qpd.DataFrame(df)
q_res = qpd.dropna(q_df, return_type="q")

qpd.print(q_res)

pd_res = df.dropna()
assert verify_correctness(pd_res, qpd.dropna(q_df, return_type="p"))


┌─────┬──────┐
│ a   │ b    │
├─────┼──────┤
│ 1.0 │ 10.0 │
└─────┴──────┘


In [22]:
# dropna where no nulls exist
# Expected: Table remains unchanged
df = pd.DataFrame({
    "a": [1, 2],
    "b": [3, 4]
})

q_df = qpd.DataFrame(df)
q_res = qpd.dropna(q_df, return_type="q")

qpd.print(q_res)

pd_res = df.dropna()
assert verify_correctness(pd_res, qpd.dropna(q_df, return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ 3 │
│ 2 │ 4 │
└───┴───┘


In [23]:
# dropna on empty table
# Expected: Empty table is returned
df = pd.DataFrame(columns=["a", "b"])

q_df = qpd.DataFrame(df)
q_res = qpd.dropna(q_df, return_type="q")

qpd.print(q_res)

pd_res = df.dropna()
assert verify_correctness(pd_res, qpd.dropna(q_df, return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
└───┴───┘


In [24]:
# dropna on table with all rows containing nulls
# Expected: Empty table is returned
df = pd.DataFrame({
    "a": [None, None],
    "b": [None, None]
})

q_df = qpd.DataFrame(df)
q_res = qpd.dropna(q_df, return_type="q")

qpd.print(q_res)

pd_res = df.dropna()
assert verify_correctness(pd_res, qpd.dropna(q_df, return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
└───┴───┘


In [25]:
# fillna with single-column dictionary
# Expected: Nulls in specified column are filled
df = pd.DataFrame({
    "a": [1, None, 3],
    "b": [10, 20, 30]
})

q_df = qpd.DataFrame(df)
q_res = qpd.fillna(q_df, {"a": 0}, return_type="q")

qpd.print(q_res)

pd_res = df.fillna({"a": 0})
assert verify_correctness(pd_res, qpd.fillna(q_df, {"a": 0}, return_type="p"))


┌─────┬────┐
│ a   │ b  │
├─────┼────┤
│ 1.0 │ 10 │
│ 0.0 │ 20 │
│ 3.0 │ 30 │
└─────┴────┘


In [26]:
# fillna with multiple columns dictionary
# Expected: Each column is filled independently
df = pd.DataFrame({
    "a": [1, None, 3],
    "b": [None, 2, None]
})

q_df = qpd.DataFrame(df)
q_res = qpd.fillna(q_df, {"a": 0, "b": 9}, return_type="q")

qpd.print(q_res)

pd_res = df.fillna({"a": 0, "b": 9})
assert verify_correctness(pd_res, qpd.fillna(q_df, {"a": 0, "b": 9}, return_type="p"))


┌─────┬─────┐
│ a   │ b   │
├─────┼─────┤
│ 1.0 │ 9.0 │
│ 0.0 │ 2.0 │
│ 3.0 │ 9.0 │
└─────┴─────┘


In [27]:
# fillna with string values
# Expected: String nulls are replaced correctly
df = pd.DataFrame({
    "a": ["x", None, "z"],
    "b": [1, 2, 3]
})

q_df = qpd.DataFrame(df)
q_res = qpd.fillna(q_df, {"a": "y"}, return_type="q")

qpd.print(q_res)

pd_res = df.fillna({"a": "y"})
assert verify_correctness(pd_res, qpd.fillna(q_df, {"a": "y"}, return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
│ x │ 1 │
│ y │ 2 │
│ z │ 3 │
└───┴───┘


In [28]:
# fillna where no nulls exist
# Expected: Table remains unchanged
df = pd.DataFrame({
    "a": [1, 2],
    "b": [3, 4]
})

q_df = qpd.DataFrame(df)
q_res = qpd.fillna(q_df, {"a": 0, "b": 0}, return_type="q")

qpd.print(q_res)

pd_res = df.fillna({"a": 0, "b": 0})
assert verify_correctness(pd_res, qpd.fillna(q_df, {"a": 0, "b": 0}, return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ 3 │
│ 2 │ 4 │
└───┴───┘


In [29]:
# fillna with empty dictionary
# Expected: Table is returned unchanged
df = pd.DataFrame({
    "a": [1, None],
    "b": [2, None]
})

q_df = qpd.DataFrame(df)
q_res = qpd.fillna(q_df, {}, return_type="q")

qpd.print(q_res)

pd_res = df.fillna({})
assert verify_correctness(pd_res, qpd.fillna(q_df, {}, return_type="p"))


┌─────┬─────┐
│ a   │ b   │
├─────┼─────┤
│ 1.0 │ 2.0 │
│ nan │ nan │
└─────┴─────┘


In [30]:
# fillna on empty table
# Expected: Empty table is returned
df = pd.DataFrame(columns=["a", "b"])

q_df = qpd.DataFrame(df)
q_res = qpd.fillna(q_df, {"a": 0}, return_type="q")

qpd.print(q_res)

pd_res = df.fillna({"a": 0})
assert verify_correctness(pd_res, qpd.fillna(q_df, {"a": 0}, return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
└───┴───┘


In [31]:
# fillna with missing column in dictionary
# Expected: RuntimeError is raised
df = pd.DataFrame({
    "a": [1, None]
})

q_df = qpd.DataFrame(df)

try:
    qpd.fillna(q_df, {"b": 0})
    raise AssertionError("Expected RuntimeError")
except RuntimeError:
    pass


In [32]:
# fillna with non-dictionary input
# Expected: RuntimeError is raised
df = pd.DataFrame({
    "a": [1, None]
})

q_df = qpd.DataFrame(df)

try:
    qpd.fillna(q_df, "a")
    raise AssertionError("Expected RuntimeError")
except RuntimeError:
    pass


In [33]:
# remove_duplicates on DataFrame with duplicate rows
# Expected: Duplicate rows are removed, first occurrence kept
df = pd.DataFrame({
    "a": [1, 1, 2, 2],
    "b": [10, 10, 20, 20]
})

q_df = qpd.DataFrame(df)
q_res = qpd.remove_duplicates(q_df, return_type="q")

qpd.print(q_res)

pd_res = df.drop_duplicates()
assert verify_correctness(pd_res, qpd.remove_duplicates(q_df, return_type="p"))


┌───┬────┐
│ a │ b  │
├───┼────┤
│ 1 │ 10 │
│ 2 │ 20 │
└───┴────┘


In [34]:
# remove_duplicates where no duplicates exist
# Expected: Table remains unchanged
df = pd.DataFrame({
    "a": [1, 2, 3],
    "b": [4, 5, 6]
})

q_df = qpd.DataFrame(df)
q_res = qpd.remove_duplicates(q_df, return_type="q")

qpd.print(q_res)

pd_res = df.drop_duplicates()
assert verify_correctness(pd_res, qpd.remove_duplicates(q_df, return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ 4 │
│ 2 │ 5 │
│ 3 │ 6 │
└───┴───┘


In [35]:
# remove_duplicates on DataFrame with all rows duplicated
# Expected: Single unique row remains
df = pd.DataFrame({
    "a": [1, 1, 1],
    "b": [2, 2, 2]
})

q_df = qpd.DataFrame(df)
q_res = qpd.remove_duplicates(q_df, return_type="q")

qpd.print(q_res)

pd_res = df.drop_duplicates()
assert verify_correctness(pd_res, qpd.remove_duplicates(q_df, return_type="p"))

┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ 2 │
└───┴───┘


In [36]:
# remove_duplicates on empty DataFrame
# Expected: Empty table is returned
df = pd.DataFrame(columns=["a", "b"])

q_df = qpd.DataFrame(df)
q_res = qpd.remove_duplicates(q_df, return_type="q")

qpd.print(q_res)

pd_res = df.drop_duplicates()
assert verify_correctness(pd_res, qpd.remove_duplicates(q_df, return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
└───┴───┘


In [37]:
# remove_duplicates on DataFrame with mixed dtypes
# Expected: Duplicate rows are removed correctly
df = pd.DataFrame({
    "a": [1, 1, 2],
    "b": ["x", "x", "y"]
})

q_df = qpd.DataFrame(df)
q_res = qpd.remove_duplicates(q_df, return_type="q")

qpd.print(q_res)

pd_res = df.drop_duplicates()
assert verify_correctness(pd_res, qpd.remove_duplicates(q_df, return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ x │
│ 2 │ y │
└───┴───┘


In [38]:
# remove_duplicates on single-row DataFrame
# Expected: Single row is preserved
df = pd.DataFrame({
    "a": [1],
    "b": [2]
})

q_df = qpd.DataFrame(df)
q_res = qpd.remove_duplicates(q_df, return_type="q")

qpd.print(q_res)

pd_res = df.drop_duplicates()
assert verify_correctness(pd_res, qpd.remove_duplicates(q_df, return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ 2 │
└───┴───┘


## Transformation Functions
Test structural and type transformations on the DataFrame.

In [39]:
# cast int column to float
# Expected: Column is converted to float type
df = pd.DataFrame({
    "a": [1, 2, 3],
    "b": [10, 20, 30]
})

q_df = qpd.DataFrame(df)
q_res = qpd.cast(q_df, "a", "float", return_type="q")

qpd.print(q_res)

pd_res = df.copy()
pd_res["a"] = pd_res["a"].astype(float)
assert verify_correctness(pd_res, qpd.cast(q_df, "a", "float", return_type="p"))


┌─────┬────┐
│ a   │ b  │
├─────┼────┤
│ 1.0 │ 10 │
│ 2.0 │ 20 │
│ 3.0 │ 30 │
└─────┴────┘


In [40]:
# cast float column to int
# Expected: Column is converted to int (truncation behavior matches Pandas)
df = pd.DataFrame({
    "a": [1.2, 2.7, 3.0],
    "b": [1, 2, 3]
})

q_df = qpd.DataFrame(df)
q_res = qpd.cast(q_df, "a", "int", return_type="q")

qpd.print(q_res)

pd_res = df.copy()
pd_res["a"] = pd_res["a"].astype(int)
assert verify_correctness(pd_res, qpd.cast(q_df, "a", "int", return_type="p"))

┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ 1 │
│ 2 │ 2 │
│ 3 │ 3 │
└───┴───┘


In [41]:
# cast string column to symbol
# Expected: String values are converted to symbol type
df = pd.DataFrame({
    "a": ["x", "y", "z"],
    "b": [1, 2, 3]
})

q_df = qpd.DataFrame(df)
q_res = qpd.cast(q_df, "a", "s", return_type="q")

qpd.print(q_res)

pd_res = df.copy()
pd_res["a"] = pd_res["a"].astype(str)
assert verify_correctness(pd_res, qpd.cast(q_df, "a", "s", return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
│ x │ 1 │
│ y │ 2 │
│ z │ 3 │
└───┴───┘


In [42]:
# cast column containing nulls
# Expected: Null values are preserved after casting
df = pd.DataFrame({
    "a": [1, None, 3],
    "b": [4, 5, 6]
})

q_df = qpd.DataFrame(df)
q_res = qpd.cast(q_df, "a", "float", return_type="q")

qpd.print(q_res)

pd_res = df.copy()
pd_res["a"] = pd_res["a"].astype(float)
assert verify_correctness(pd_res, qpd.cast(q_df, "a", "float", return_type="p"))


┌─────┬───┐
│ a   │ b │
├─────┼───┤
│ 1.0 │ 4 │
│ nan │ 5 │
│ 3.0 │ 6 │
└─────┴───┘


In [43]:
# cast on empty DataFrame
# Expected: Empty table is returned
df = pd.DataFrame(columns=["a", "b"])

q_df = qpd.DataFrame(df)
q_res = qpd.cast(q_df, "a", "int", return_type="q")

qpd.print(q_res)

pd_res = df.copy()
pd_res["a"] = pd_res["a"].astype("int64", errors="ignore")
assert verify_correctness(pd_res, qpd.cast(q_df, "a", "int", return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
└───┴───┘


In [44]:
# cast with unsupported dtype
# Expected: RuntimeError is raised
df = pd.DataFrame({
    "a": [1, 2, 3]
})

q_df = qpd.DataFrame(df)

try:
    qpd.cast(q_df, "a", "unsupported_type")
    raise AssertionError("Expected RuntimeError")
except RuntimeError:
    pass


In [45]:
# cast on non-existent column
# Expected: RuntimeError is raised
df = pd.DataFrame({
    "a": [1, 2, 3]
})

q_df = qpd.DataFrame(df)

try:
    qpd.cast(q_df, "b", "int")
    raise AssertionError("Expected RuntimeError")
except RuntimeError:
    pass


In [46]:
# cast using q-type shorthand
# Expected: Column is cast using q shorthand type
df = pd.DataFrame({
    "a": [1, 2, 3]
})

q_df = qpd.DataFrame(df)
q_res = qpd.cast(q_df, "a", "j", return_type="q")

qpd.print(q_res)

pd_res = df.copy()
pd_res["a"] = pd_res["a"].astype(int)
assert verify_correctness(pd_res, qpd.cast(q_df, "a", "j", return_type="p"))


┌───┐
│ a │
├───┤
│ 1 │
│ 2 │
│ 3 │
└───┘


In [47]:
# drop_col with array of columns
# Expected: All specified columns are removed
df = pd.DataFrame({
    "a": [1, 2],
    "b": [3, 4],
    "c": [5, 6]
})

q_df = qpd.DataFrame(df)
q_res = qpd.drop_col(q_df, ["a", "c"], return_type="q")

qpd.print(q_res)

pd_res = df.drop(columns=["a", "c"])
assert verify_correctness(pd_res, qpd.drop_col(q_df, ["a", "c"], return_type="p"))


┌───┐
│ b │
├───┤
│ 3 │
│ 4 │
└───┘


In [48]:
# drop_col with array containing single column
# Expected: Specified column is removed
df = pd.DataFrame({
    "a": [1, 2],
    "b": [3, 4]
})

q_df = qpd.DataFrame(df)
q_res = qpd.drop_col(q_df, ["b"], return_type="q")

qpd.print(q_res)

pd_res = df.drop(columns=["b"])
assert verify_correctness(pd_res, qpd.drop_col(q_df, ["b"], return_type="p"))


┌───┐
│ a │
├───┤
│ 1 │
│ 2 │
└───┘


In [49]:
# drop_col with string input
# Expected: String is normalized to array and column is removed
df = pd.DataFrame({
    "a": [1, 2],
    "b": [3, 4]
})

q_df = qpd.DataFrame(df)
q_res = qpd.drop_col(q_df, "a", return_type="q")

qpd.print(q_res)

pd_res = df.drop(columns=["a"])
assert verify_correctness(pd_res, qpd.drop_col(q_df, "a", return_type="p"))


┌───┐
│ b │
├───┤
│ 3 │
│ 4 │
└───┘


In [50]:
# drop_col with empty array
# Expected: Table remains unchanged
df = pd.DataFrame({
    "a": [1, 2],
    "b": [3, 4]
})

q_df = qpd.DataFrame(df)
q_res = qpd.drop_col(q_df, [], return_type="q")

qpd.print(q_res)

pd_res = df.copy()
assert verify_correctness(pd_res, qpd.drop_col(q_df, [], return_type="p"))


┌───┬───┐
│ a │ b │
├───┼───┤
│ 1 │ 3 │
│ 2 │ 4 │
└───┴───┘


In [ ]:
# drop_col with all columns specified
# Expected: Resulting table has no columns and row count preserved

df = pd.DataFrame({
        "a": [1, 2, 3],
        "b": [4, 5, 6]
    })
q_df = kx.toq(df)
    
print("Testing drop_col ALL (return_type='p')...")
# This should return an empty DataFrame with index 0..2
p_res = qpd.drop_col(q_df, ["a", "b"], return_type='p')

print("Result:\n", p_res)
assert isinstance(p_res, pd.DataFrame)
assert len(p_res.columns) == 0
assert len(p_res) == 3
print("PASS: return_type='p' returned empty DataFrame with 3 rows.")
print("\nTesting drop_col ALL (return_type='q')...")
# This should return a Dictionary (PyKX workaround)
q_res = qpd.drop_col(q_df, ["a", "b"], return_type='q')

print("Result type:", type(q_res))
assert isinstance(q_res, kx.Dictionary)
print("PASS: return_type='q' returned Dictionary (empty columns repr).")

Testing drop_col ALL (return_type='p')...


NameError: name 'drop_col' is not defined

## Single Column Tables
Tests operations on single-column DataFrames.

In [ ]:
df_single = pd.DataFrame({'a': [1, 2, 3]})
q_single = qpd.DataFrame(df_single)
pd_res = df_single.rename(columns={'a': 'x'})
q_res = qpd.rename(q_single, {'a': 'x'}, return_type='p')
assert verify_correctness(pd_res, q_res)
print('Passed: Single Column Tables')


## Error Handling: Missing Columns
Tests that operations raise appropriate errors when targeting non-existent columns.

In [ ]:
df = pd.DataFrame({'a': [1]})
q_table = qpd.DataFrame(df)
try:
    qpd.drop_col(q_table, 'missing', return_type='p')
    assert False, 'Should have raised RuntimeError'
except RuntimeError:
    pass
try:
    qpd.groupby_sum(q_table, 'missing', 'a', return_type='p')
    assert False, 'Should have raised RuntimeError'
except RuntimeError:
    pass
print('Passed: Error Handling: Missing Columns')


## Error Handling: Incorrect inputs
Tests that operations raise errors for invalid input types (e.g., list instead of DataFrame).

In [ ]:
try:
    qpd.dropna([1, 2, 3], return_type='p')
    assert False, 'Should have raised ValueError/RuntimeError'
except (ValueError, RuntimeError):
    pass
print('Passed: Error Handling: Incorrect inputs')


## Null-Heavy Columns
Tests handling of columns with majorly null values.

In [ ]:
df_nulls = pd.DataFrame({'a': [None]*100, 'b': range(100)})
q_nulls = qpd.DataFrame(df_nulls)
pd_res = df_nulls.dropna()
q_res = qpd.dropna(q_nulls, return_type='p')
assert verify_correctness(pd_res, q_res)
pd_res = df_nulls[df_nulls['a'].notna()]
q_res = qpd.dropna_col(q_nulls, 'a', return_type='p')
assert verify_correctness(pd_res, q_res)
print('Passed: Null-Heavy Columns')


## Test print Edge Cases
Test print with empty tables and large row counts.

In [ ]:
print('Testing qpd.print with empty table:')
empty_df = pd.DataFrame({'a': [], 'b': []})
q_empty = qpd.DataFrame(empty_df)
qpd.print(q_empty)

print('\nTesting qpd.print with large table (n=10):')
large_df = pd.DataFrame({'x': range(1000), 'y': range(1000, 2000)})
q_large = qpd.DataFrame(large_df)
qpd.print(q_large, n=10)

print('\nEdge case tests passed')

## Mixed Data Types
Tests casting operations on mixed-type DataFrames.

In [ ]:
df_mixed = pd.DataFrame({'a': [1, 2], 'b': ['x', 'y'], 'c': [1.1, 2.2]})
q_mixed = qpd.DataFrame(df_mixed)
pd_res = df_mixed.astype({'a': 'float64'})
q_res = qpd.cast(q_mixed, 'a', 'float64', return_type='p')
assert verify_correctness(pd_res, q_res)
print('Passed: Mixed Data Types')


## Wide Tables
Tests operations on tables with many columns (500+).

In [ ]:
wide_df = pd.DataFrame({f'c{i}': [i] for i in range(500)})
q_wide = qpd.DataFrame(wide_df)
pd_res = wide_df.drop(columns=['c250'])
q_res = qpd.drop_col(q_wide, 'c250', return_type='p')
assert verify_correctness(pd_res, q_res)
print('Passed: Wide Tables')


## Duplicate Column Names
Tests handling (or graceful failure) of duplicate column names.

In [ ]:
df_dup = pd.DataFrame([[1, 2]], columns=['a', 'a'])
try:
    q_dup = qpd.DataFrame(df_dup)
    q_res = qpd.dropna(q_dup, return_type='p')
    print('Passed: Duplicate Column Names')
except:
    print('Expected Failure: Duplicate Column Names (or passed with auto-renaming)')


## Return Type Validation
Tests strict enforcement of 'p' and 'q' return types and error handling for invalid types.

In [ ]:
df_test = pd.DataFrame({'a': [1, 2, None], 'b': ['x', 'y', 'z']})
q_test = kx.toq(df_test)

res_p = qpd.dropna(q_test, return_type='p')
assert isinstance(res_p, pd.DataFrame)

res_q = qpd.dropna(q_test, return_type='q')
assert isinstance(res_q, (kx.Table, kx.KeyedTable))

try:
    qpd.dropna(q_test, return_type='pandas')
    assert False, "Should raise ValueError for 'pandas'"
except (ValueError, RuntimeError) as e:
    assert "Invalid return_type" in str(e)

res_sum_p = qpd.groupby_sum(q_test, 'b', 'a', return_type='p')
assert isinstance(res_sum_p, pd.DataFrame)

res_apply_p = qpd.apply(q_test, 'count', axis=0, return_type='p')
assert isinstance(res_apply_p, (pd.DataFrame, pd.Series, dict))

print('Passed: Return Type Validation')


## Introspection: dtypes
Tests dtypes.

In [ ]:
# Basic Functional & Return Type Test
df = pd.DataFrame({'a': [1, 2], 'b': ['x', 'y']})
q_table = kx.toq(df)

# Test q return
res_q = qpd.dtypes(q_table, return_type='q')
assert isinstance(res_q, (kx.Table, kx.KeyedTable)), "return_type='q' mismatch"

# Test p return
res_p = qpd.dtypes(q_table, return_type='p')
assert isinstance(res_p, pd.DataFrame), "return_type='p' mismatch"
assert 'a' in res_p.index or 'a' in res_p.iloc[:, 0].values

# Empty DataFrame
df_empty = pd.DataFrame(columns=['a', 'b'])
q_empty = qpd.DataFrame(df_empty)
res = qpd.dtypes(q_empty, return_type='p')
assert len(res) == 2

# Null columns
df_null = pd.DataFrame({'a': [None, None], 'b': [1, 2]})
q_null = qpd.DataFrame(df_null)
res = qpd.dtypes(q_null, return_type='p')
assert len(res) == 2

print('Passed: dtypes Introspection')

## Indexing Edge Cases
Tests for `loc` and `iloc` error handling and boundary conditions.

In [ ]:
df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
q_table = qpd.DataFrame(df)

# Unknown column in loc
try:
    qpd.loc(q_table, cols='z', return_type='p')
    print("Failed to raise error for unknown col")
except (ValueError, RuntimeError) as e:
    print(f"Caught expected error for unknown col: {e}")

# Empty selection iloc
res = qpd.iloc(q_table, rows=[], return_type='p')
assert len(res) == 0, f"Expected 0 rows, got {len(res)}"
print("Empty selection verified")

# Out of bounds slice (iloc)
res = qpd.iloc(q_table, rows=slice(0, 100), return_type='p')
assert len(res) == 3, f"Expected 3 rows (clipped), got {len(res)}"
print("Out of bounds slice verified")

## Test print Edge Cases
Test print with empty tables and large row counts.

In [ ]:
print('Testing qpd.print with empty table:')
empty_df = pd.DataFrame({'a': [], 'b': []})
q_empty = qpd.DataFrame(empty_df)
qpd.print(q_empty)

print('\nTesting qpd.print with large table (n=10):')
large_df = pd.DataFrame({'x': range(1000), 'y': range(1000, 2000)})
q_large = qpd.DataFrame(large_df)
qpd.print(q_large, n=10)

print('\nEdge case tests passed')

## Test I/O Edge Cases
Test I/O with special characters, empty tables, and complex paths.

In [ ]:
edge_io_df = pd.DataFrame({
    'text': ['hello, world', 'newline\ntest', 'quotes"test"', 'symbols!@#$%^&*()']
})
q_edge_io = qpd.DataFrame(edge_io_df)
edge_csv = 'edge_test_io.csv'

print("Testing I/O with special characters...")
qpd.to_csv(q_edge_io, edge_csv)
loaded_edge = qpd.from_csv(edge_csv, return_type='p')
print("Loaded special characters:")
print(loaded_edge)

empty_io = qpd.DataFrame(pd.DataFrame({'a': [], 'b': []}))
empty_csv = 'empty_io.csv'
print("\nTesting I/O with empty table...")
qpd.to_csv(empty_io, empty_csv)
loaded_empty = qpd.from_csv(empty_csv, return_type='p')
print(f"Loaded empty table shape: {loaded_empty.shape}")

for f in [edge_csv, empty_csv]:
    if os.path.exists(f): os.remove(f)
print("\nEdge case I/O tests completed.")